# Basic ML Ops For REST APIs With Docker And Jenkins

> by Thom Ives, Ph.D.

![Integrated Machine Learning & AI](https://dagshub.com/ThomIves/PyTorch_Journey/raw/aefdfaa0c20cdff3c13e8238f1cb93bec72eb461/IMLAI_1700x500_Inv.png)

## Overview

1. Develop Your REST API Using FastAPI On Your **Dev** System
1. Ensure That You Have Docker Installed And Install It If You Do Not
1. Install The Latest Jenkins LTS Version On Your **Host** System Using The "Generic Java Package (.war)" Method
1. Create A GitHub Repo To Hold Your Code
    1. Your Dev System Will Hold A Clone
    1. Make Changes / Improvements And Commit And Push From Your **Dev** System
1. Clone Your Initial Code To Your **Host** System In The Correct Location For Working With Jenkins
1. Create A Docker Image And Run Its Container For Your API
    1. Write Your Dockerfile That Will Create Your Image
    1. Build Your Image
    1. Run Your Image In A Docker Container With A Directory From Your Machine Mounted To Your Container
1. Create A Jenkins Pipeline That Will:
    1. Update The Code On Your **Host** System
    1. When You Push Changes From Your **Dev** System
    1. THIS Is Our SIMPLE CI-CD For REST APIs

## Develop Your REST API Using FastAPI On Your **Dev** System

There is an initial repo for this [HERE](https://dagshub.com/ThomIves/Basic_Model_Serving_With_FastAPI) called "Basic_Model_Serving_With_FastAPI".

Let's go look at it.

## Ensure That You Have Docker Installed

Please see my [repository](https://dagshub.com/ThomIves/Docker_Mastery) covering my studies on Docker, which include how to install it.

## Install The Latest Jenkins LTS Version On Your HOST System<br>Using The "Generic Java Package (.war)" Method

Go [HERE](https://www.jenkins.io/download/) and read carefully. 
The simplest and recommended way to install Jenkins is with "Generic Java Package (.war)" described on that page.

When you reboot, and you want to restart jenkins, open a terminal and type `java -jar jenkins.war`.
Better yet, put that in a shell file in Linux (batch file on Windows, and I am guessing shell file on Mac) and name it something clear to you.
I have a start-jenkins.sh file with the contents `java -jar jenkins.war`.
You could then even call this start Jenkins type file from one of your startup scripts.
When you want to use Jenkins, you open your browser and go to `localhost:8080 or whatever you've set your port for Jenkins to run on. 
How do you specify a different port if you need 8080 for something else?
You use `java -jar jenkins.war --httpPort=[port number]`.

[Install And Run Jenkins With war File](https://www.jenkins.io/doc/book/installing/war-file/)
[How To Change Port For Jenkins](https://phoenixnap.com/kb/jenkins-change-port)

But Wait!
What to do about that terminal that started Jenkins?
I might close it, and then I interrupt my work!
And where is best to store my jenkins.war file?
You do you, but I put my jenkins.war file in `/usr/share/jenkins`.
I also added `nohup` to the front of my shell script command. 
So now that shell script has `nohup java -jar /usr/share/jenkins/jenkins.war`.
I can then close my terminal after I start it manually, or not worry about it after it starts from my `~/.bashrc` file.

## Create A GitHub Repo To Hold Your Code

![GitHub Repo For CI-CD](GitHub_Repo.png)

### Your Dev System Will Hold A Clone

__Show Development File Manager__

__Make Changes / Improvements And Commit And Push From Your **Dev** System__

## Clone Your Initial Code To Your HOST System In The Correct Location For Working With Jenkins

__Show Jenkins Clone Of Code__

__We Also Build And Run Our Docker Image And Container From The Host__

## Create A Docker Image And Run Its Container For Your API

### Write Your Dockerfile That Will Create Your Image

FROM python:3.8

WORKDIR /src

COPY ./requirements.txt /src
COPY ./main.py /src
COPY ./Linear_Regression_Model.pkl /src

RUN pip install -r requirements.txt

EXPOSE 8000:8000

CMD ["uvicorn", "Run_Model_API:app", "--reload", "--host", "0.0.0.0"]

### Build Your Image

`$ sudo docker image build . -t api:1`

### Run Your Image In A Docker Container With A Directory From Your Machine Mounted To Your Container

`$ sudo docker run -d -p 8000:8000 -v /home/thom/.jenkins/workspace/Model_Serving_With_FastAPI/src:/src api:1`

## A Simpler Example For CI CD

### The FastAPI REST API Code - main.py

In [ ]:
# main.py
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
async def root():
    return {"message": "This is a simple API."}


### The Dockefile

In [ ]:
FROM python:3.8

WORKDIR /src

COPY ./requirements.txt /src
COPY ./main.py /src

RUN pip install -r requirements.txt

EXPOSE 8000:8000

CMD ["uvicorn", "main:app", "--reload", "--host", "0.0.0.0"]


### The requirements.txt File

In [ ]:
fastapi
uvicorn

### The Jenkins File

In [ ]:
pipeline {
    agent any 
    stages {
        stage ("Go To Git Directory"){
            steps {
                sh "cd /home/thom/.jenkins/workspace/Docker_Jenkins_Cloud_API"
            }
        }
        stage ("Pull Changes From Repo"){
            steps {
                script{
                    checkout([$class: 'GitSCM', branches: [[name: 'master']], extensions: [], userRemoteConfigs: [[url: 'https://github.com/ThomIves/Docker_Jenkins_Cloud_API.git']]])
                }
            }
        }
        stage ("Report Status"){
            steps {
                echo "Updates Complete."
            }
        }
    }
}


## Summary

1. Develop REST API With FastAPI
1. Dockerize Your API Or Not - Serve It Either Way From A Host - On Prem Or From Cloud VM
1. Setup Jenkins To Watch Your Cloud Git Repo To Update Your Host Files
1. Changes Will Pass To Controlling Directory On Host
1. Uvicorn Will See Changes And Restart